# Inference and Evaluation for RoBERTa NLI Model

## Install Dependencies

In [6]:
!pip install transformers pandas torch


## Load Model and Tokenizer

In [7]:
import pandas as pd
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import numpy as np
import shutil

# Load the dataset

model = RobertaForSequenceClassification.from_pretrained("/kaggle/input/roberta_nli_model/transformers/default/1/model")
tokenizer = RobertaTokenizer.from_pretrained("/kaggle/input/roberta_nli_model/transformers/default/1/model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

## Step 1: Evaluate on dev.csv

In [9]:
dev_raw = pd.read_csv("/kaggle/input/nlu-data-file/dev.csv")
encodings = tokenizer(list(dev_raw['premise']), list(dev_raw['hypothesis']),
                      truncation=True, padding=True, max_length=512)

input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])
labels = torch.tensor(dev_raw['label'].values)
dev_dataset = TensorDataset(input_ids, attention_mask, labels)
dev_loader = DataLoader(dev_dataset, batch_size=32)

dev_preds, all_labels = [], []
with torch.no_grad():
    for batch in tqdm(dev_loader):
        b_input_ids, b_mask, b_labels = [t.to(device) for t in batch]
        outputs = model(input_ids=b_input_ids, attention_mask=b_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        dev_preds.extend(preds)
        all_labels.extend(b_labels.cpu().numpy())

# Print evaluation scores
print("Accuracy:", accuracy_score(all_labels, dev_preds))
print("F1 score:", f1_score(all_labels, dev_preds))

# Save as Codabench CSV format
codabench_df = pd.DataFrame({"prediction": dev_preds})
codabench_df.to_csv("predictions.csv", index=False)
shutil.copy("predictions.csv", "predictions.csv.predict")
!zip predictions.csv.predict.zip predictions.csv.predict
print("Finished!")

100%|██████████| 211/211 [00:30<00:00,  6.86it/s]

Accuracy: 0.8873218527315915
F1 score: 0.8918341171440787
  adding: predictions.csv.predict (deflated 90%)


Finished!


## Step 2: Predict on test.csv and save output

In [11]:
test_raw = pd.read_csv("/kaggle/input/nlu-data-file/test.csv")
encodings = tokenizer(list(test_raw['premise']), list(test_raw['hypothesis']),
                      truncation=True, padding=True, max_length=512)

input_ids = torch.tensor(encodings['input_ids'])
attention_mask = torch.tensor(encodings['attention_mask'])
dataset = TensorDataset(input_ids, attention_mask)
loader = DataLoader(dataset, batch_size=32)


test_preds = []

with torch.no_grad():
    for batch in tqdm(loader):
        b_input_ids, b_mask = [t.to(device) for t in batch]
        outputs = model(input_ids=b_input_ids, attention_mask=b_mask)
        preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

# Save prediction CSV for Blackboard
test_df = pd.DataFrame({"prediction": test_preds})
test_df.to_csv("Group_16_C.csv", index=False)

print("Group_16_C.csv Saved")

100%|██████████| 104/104 [00:07<00:00, 14.82it/s]

Group_16_C.csv Saved
